In [5]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import binascii
import syft as sy

# import sys
# sys.path.insert(0, <path to dirFoo>)

# import os
# import sys
# sys.path.append(os.getcwd())
from app.main.processes import processes


# import os.path, sys
# sys.path.append(os.path.dirname(__file__))



from syft.serde.serde import serialize, deserialize



# async def test_fl_process(self):
#         """ 1 - Host Federated Training """
#         # Plan Functions
@sy.func2plan(args_shape=[(1,), (1,), (1,)])
def foo_1(x, y, z):
    a = x + x
    b = x + z
    c = y + z
    return c, b, a

@sy.func2plan(args_shape=[(1,), (1,), (1,)])
def foo_2(x, y, z):
    a = x + x
    b = x + z
    return b, a

@sy.func2plan(args_shape=[(1,), (1,)])
def avg_plan(x, y):
    result = x + y / 2
    return result

# Plan Model
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__(id="my-model")
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)
        self.fc3 = nn.Linear(2, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=0)

model = Net()
model.build(th.tensor([1.0, 2]))

# Serialize plans / protocols and model
serialized_plan_method_1 = binascii.hexlify(serialize(foo_1)).decode()
#         serialized_plan_method_2 = binascii.hexlify(serialize(foo_2)).decode()
serialized_avg_plan = binascii.hexlify(serialize(avg_plan)).decode()
serialized_plan_model = binascii.hexlify(serialize(model)).decode()

# As mentioned at federated learning roadmap.
# We're supposed to set up client / server configs
client_config = {
    "name": "my-federated-model",
    "version": "0.1.0",
    "batch_size": 32,
    "lr": 0.01,
    "optimizer": "SGD",
}

server_config = {
    "max_workers": 100,
    "pool_selection": "random",  # or "iterate"
    "num_cycles": 5,
    "do_not_reuse_workers_until_cycle": 4,
    "cycle_length": 8 * 60 * 60,  # 8 hours
    "minimum_upload_speed": 2,  # 2 mbps
    "minimum_download_speed": 4,  # 4 mbps
}

# "federated/host-training" request body
host_training_message = {
    "type": "federated/host-training",
    "data": {
        "model": serialized_plan_model,
        "plans": {
            "foo_1": serialized_plan_method_1,
#             "foo_2": serialized_plan_method_2,
        },
        "protocols": {"protocol_1": "serialized_protocol_mockup"},
        "averaging_plan": serialized_avg_plan,
        "client_config": client_config,
        "server_config": server_config,
    },
}

# Send host_training message
#         response = await send_ws_message(host_training_message)
#         self.assertEqual(response, {"status": "success"})


processes.create_process(serialized_plan_model, serialized_plan_method_1, client_config, server_config, serialized_avg_plan)







RuntimeError: No application found. Either work inside a view function or push an application context. See http://flask-sqlalchemy.pocoo.org/contexts/.

In [2]:
pwd

'/Users/mshang/Desktop/dev_env/openMined/PyGrid/gateway'